In [ ]:
%pip install suite2p

In [ ]:
import os
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np

import s2p_vis_func
import s2p_con_func
import s2p_plot_rois_activity_funcs

import suite2p

# User Variables
Please input the required variables below

threshold_scaling: a parameter for the suite2p pre-processing that determines the threshold for what the algorithm considers to be an ROI. You can input as many values as you like into the list and the program will output the visualizations for all of them. We recommend values from around 0.4 to 2.0. The higher the number, the higher the threshold for what is considered an ROI and thus, less ROI's will be detected

fname: The file name of the data you wish to process. Please remember to place it inside quotes as shown below

fdir: The name of the path, or directory, of the data file you wish to process. IMPORTANT: keep the r before the quotes and leave off the last backslash

tseries_start_end: the timescale you wish to see for the visualization of the ROI traces. If you type None in the brackets the whole session will be processed

rois_to_plot: can either be an integer with the number of ROI's you wish to see plotted, a list with the specific ROI numbers, or if you type None all of the ROI's will be shown

In [ ]:
ops = suite2p.default_ops()
ops['fs'] = 13 # sampling rate of recording, determines binning for cell detection
ops['tau'] = 1.25 # timescale of gcamp to use for deconvolution
ops['threshold_scaling'] =  [0.6, 0.8] # set however many desired threshold_scaling values you wish to test

fname = '59879-0 666um 200pwr 10xobj 4frm avg suc70 quin30-000'
fdir =  r'C:\Users\Alex_dl\Documents\GitHub\NAPE_imaging_postprocess\napeca_post\sample_data'
tseries_start_end = [0, 100]
rois_to_plot = 50

In [ ]:
import matplotlib as mpl
mpl.rcParams.update({
    'axes.spines.left': True,
    'axes.spines.bottom': True,
    'axes.spines.top': False,
    'axes.spines.right': False,
    'legend.frameon': False,
    'figure.subplot.wspace': .01,
    'figure.subplot.hspace': .01,
    'figure.figsize': (18, 13),
    'ytick.major.left': True,
})
jet = mpl.cm.get_cmap('jet')
jet.set_bad(color='k')

In [ ]:
db = {
    'data_path': [fdir],
}
print(db)

In [ ]:
output_ops={}
output_ops['save_path'] = db['data_path'][0]
base_dir = output_ops['save_path']
for threshold_scaling_value in ops['threshold_scaling']:
    ops['threshold_scaling'] = threshold_scaling_value
    
    #deals with creating and saving outputs into correct folders
    output_ops['save_path'] = os.path.join(base_dir, f'{fname}_{threshold_scaling_value}')
    ops['save_folder'] = f'{fname}_{threshold_scaling_value}'
    output_ops = suite2p.run_s2p(ops=ops, db=db)
    
    output_fig_dir = os.path.join(base_dir, 's2p_analysis')

    #functions that output the Registration visualizations(only outputs once)
    if not os.path.exists(output_fig_dir):
        s2p_vis_func.check_exist_dir(output_fig_dir)
        s2p_vis_func.reg_ref_imgs(output_ops, output_fig_dir)
        s2p_vis_func.reg_displacement_vis(output_ops, output_fig_dir)
    
    #functions that output the ROI detections visualizations
    s2p_vis_func.roi_mask_vis(output_ops, output_fig_dir, threshold_scaling_value)
    s2p_vis_func.roi_trace_vis(output_ops, output_fig_dir, threshold_scaling_value)

    #functions that output the post-processed roi_plots based on the desired number of ROI's
    path_dict = {}
    path_dict['threshold_scaling_value'] = threshold_scaling_value
    path_dict['s2p_dir'] = os.path.join(fdir, f'{fname}_{threshold_scaling_value}', 'plane0')
    path_dict = s2p_plot_rois_activity_funcs.define_paths_roi_plots(path_dict, tseries_start_end, rois_to_plot, output_fig_dir)
    s2p_data_dict = s2p_plot_rois_activity_funcs.load_s2p_data_roi_plots(path_dict)
    plot_vars = s2p_plot_rois_activity_funcs.plotting_rois(s2p_data_dict, path_dict)
    s2p_plot_rois_activity_funcs.masks_init(plot_vars, s2p_data_dict)
    s2p_plot_rois_activity_funcs.contour_plot(s2p_data_dict, path_dict, plot_vars)
    s2p_plot_rois_activity_funcs.time_series_plot(s2p_data_dict, path_dict, plot_vars)